1) Fit-predict https://www.kaggle.com/rahulsah06/titanic?select=train.csv

2) Feature importance

3) Loss function & optimization (философия)

4) Кластеризация

5) Работа с неструктурированными данными (tf-idf, word2vec), эмбеддинги